In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import random
import pandas as pd
from time import sleep

In [2]:
url_to_parce = 'https://www.mk.ru/news/'
news_urls = set() # all urls to links
news_mkru = [] # data for all news

In [3]:
def analyze_page(url0):
    """
    Returns a tuple with 
    url0, time_and_date, tags, title_text, full_text
    Parameters:
    url0 is a link to the news (string)
    """

    try:
        page0 = requests.get(url0)
    except:
        return None, None, None, None, None
    
        
    soup0 = BeautifulSoup(page0.text, 'html')

    time_tag  = soup0.find('time', class_ = 'meta__text')             
    try:    
        time_and_date = time_tag.get_text(strip=True)
    except:
        time_and_date = None
    
    tag_elements = soup0.find_all('a', class_='article__tag-item')
    try:
        tags = [tag.get_text() for tag in tag_elements]
    except:
        tags = None
        
    title = (soup0.find('h1', class_ =  'article__title'))
    try:
        title_text = title.text
    except:
        title_text = None

    text_contents = (soup0.find('div', class_ =  'article__body'))
    full_text = text_contents.text

    return url0, time_and_date, tags, title_text, full_text

In [4]:
for year in tqdm(range(1998, 2023)):
    for month in range(1, 12):
        for day in range(1, 31):

            url = url_to_parce + str(year)+ '/' + str(month) + '/' + str(day) + '/'
                
            try:
                page = requests.get(url)
            except:
                continue
            if page.status_code == 429: #Received a 429 Too Many Requests status.
                #print('429')
                sleep(100) # Waiting before retrying...
                continue
                    
            soup = BeautifulSoup(page.text, 'html.parser')
            for link in soup.find_all('a', class_='news-listing__item-link'):
                news_urls.add(link.get('href'))

            sleep(2)

 64%|██████▍   | 16/25 [3:25:44<2:01:53, 812.57s/it]

In [5]:
print(news_urls)

set()


In [26]:
len(news_urls)

df1 = pd.DataFrame(news_urls)
df1.to_excel('urls_mkru_archive.xlsx')

In [ ]:
for link in tqdm(news_urls):
    res = analyze_page(link)
    # print(res)
    news_mkru.append(res)
    sleep(2)


In [ ]:
df = pd.DataFrame(news_mkru)
df.columns = ['url','date and time', 'tag', 'title', 'text']
df.to_excel('parsed_news_mkru_archive.xlsx')